In [1]:
import cv2

# ========== Cấu hình ==========
video_path = 'vipham4.mp4'
output_path = 'out.mp4'
display_width = 800  # Kích thước hiển thị khung chọn điểm

# ========== Mở video và lấy frame đầu ==========
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
if not ret:
    print("❌ Không thể đọc video.")
    cap.release()
    exit()

# ========== Resize frame để hiển thị ==========
scale = display_width / frame.shape[1]
resized_frame = cv2.resize(frame, (0, 0), fx=scale, fy=scale)

points = []  # Lưu 2 điểm tạo thành 1 đoạn thẳng

# ========== Hàm xử lý chuột ==========
def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN and len(points) < 2:
        real_point = (int(x / scale), int(y / scale))
        points.append(real_point)

        # Vẽ điểm
        cv2.circle(resized_frame, (x, y), 5, (0, 255, 0), -1)

        # Nếu đã có đủ 2 điểm thì vẽ đường thẳng
        if len(points) == 2:
            pt1 = tuple([int(i * scale) for i in points[0]])
            pt2 = tuple([int(i * scale) for i in points[1]])
            cv2.line(resized_frame, pt1, pt2, (0, 0, 255), 2)

        cv2.imshow("Chọn 2 điểm", resized_frame)

# ========== Hiển thị cửa sổ chọn điểm ==========
print("🖱️ Click chuột trái để chọn đúng 2 điểm.")
cv2.imshow("Chọn 2 điểm", resized_frame)
cv2.setMouseCallback("Chọn 2 điểm", click_event)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ========== Kiểm tra số điểm ==========
if len(points) != 2:
    print("❌ Bạn chưa chọn đủ 2 điểm.")
    cap.release()
    exit()

# ========== In ra tọa độ đoạn thẳng ==========
line = tuple(points)
print("✅ Đoạn thẳng đã chọn:", line)

# ========== Chuẩn bị ghi video ==========
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# ========== Vẽ đoạn thẳng lên từng frame ==========
while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.line(frame, line[0], line[1], (0, 0, 255), 2)
    out.write(frame)

cap.release()
out.release()
print("🎉 Xuất video thành công:", output_path)



🖱️ Click chuột trái để chọn đúng 2 điểm.
✅ Đoạn thẳng đã chọn: ((1176, 763), (691, 758))
🎉 Xuất video thành công: out.mp4


In [2]:
import cv2
import numpy as np
from ultralytics import YOLO

In [3]:
def detect_traffic_light_color1(roi):
    # Chia ROI thành 3 vùng theo chiều dọc (đỏ, vàng, xanh)
    height = roi.shape[0]
    section_height = height // 3
    red_zone = roi[0:section_height, :]
    yellow_zone = roi[section_height:2*section_height, :]
    green_zone = roi[2*section_height:, :]

    # Xử lý từng vùng màu độc lập
    def get_color_intensity(zone, lower_hsv, upper_hsv):
        hsv = cv2.cvtColor(zone, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower_hsv, upper_hsv)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))  # Khử nhiễu
        return cv2.countNonZero(mask)

    # Định nghĩa dải màu HSV (tối ưu cho đèn giao thông)
    red_intensity = get_color_intensity(
        red_zone, 
        lower_hsv=np.array([0, 70, 120]), 
        upper_hsv=np.array([10, 255, 255])
    ) + get_color_intensity(
        red_zone,
        lower_hsv=np.array([160, 70, 120]),
        upper_hsv=np.array([180, 255, 255])
    )

    yellow_intensity = get_color_intensity(
        yellow_zone,
        lower_hsv=np.array([20, 70, 120]),
        upper_hsv=np.array([30, 255, 255])
    )

    green_intensity = get_color_intensity(
        green_zone,
        lower_hsv=np.array([40, 50, 50]),
        upper_hsv=np.array([90, 255, 255])
    )

    # Ngưỡng kích hoạt (có thể điều chỉnh)
    min_intensity = 30
    active_colors = []
    if red_intensity > min_intensity:
        active_colors.append(("red", red_intensity))
    if yellow_intensity > min_intensity:
        active_colors.append(("yellow", yellow_intensity))
    if green_intensity > min_intensity:
        active_colors.append(("green", green_intensity))

    # Logic xác định trạng thái đèn
    color_map = {
        "red": (0, 0, 255),
        "yellow": (0, 255, 255),
        "green": (0, 255, 0)
    }

    if not active_colors:
        return (255, 255, 255), "off"  # Đèn tắt

    # Ưu tiên màu có cường độ cao nhất
    dominant_color = max(active_colors, key=lambda x: x[1])[0]

    # Kiểm tra tính hợp lý dựa trên vị trí vật lý
    if dominant_color == "red" and red_intensity < 2 * max(yellow_intensity, green_intensity):
        return (255, 255, 255), "ambiguous"  # Trường hợp nghi ngờ

    return color_map[dominant_color], dominant_color

In [4]:

import json
from ultralytics import YOLO
import cv2

model = YOLO("yolov8n.pt")  # Hoặc model custom của bạn

# Mở video
cap = cv2.VideoCapture("out.mp4")
output_path = "end.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 30, (int(cap.get(3)), int(cap.get(4))))

detection_log = []
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Nhận diện tất cả các lớp cần thiết trong 1 lần
    results = model(frame, classes=[2, 3, 5, 7, 9])  # Car, motorbike, bus, truck, traffic light

    annotated_frame = frame.copy()

    vehicles = []
    traffic_light_color = "unknown"

    for box in results[0].boxes:
        cls_id = int(box.cls[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        if cls_id in [2, 3, 5, 7]:  # Vehicle
            vehicles.append({
                "x1": x1, "y1": y1, "x2": x2, "y2": y2,
                "class_id": cls_id
            }) 
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(annotated_frame, f"Vehicle: {cls_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        elif cls_id == 9:  # Traffic light
            roi = frame[y1:y2, x1:x2]
            box_color, color_name = detect_traffic_light_color1(roi)
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), box_color, 2)
            cv2.putText(annotated_frame, f"Light: {color_name}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, box_color, 2)
    # Ghi vào log
    detection_log.append({
    "frame": frame_idx,
    "vehicles": vehicles,
    "traffic_light": {"color": detect_traffic_light_color1(roi)}
        })
    frame_idx += 1
    # Hiển thị và ghi video
    cv2.imshow("Detection", annotated_frame)
    out.write(annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
with open("detection_log.js", "w") as f:
    f.write("const detectionData = ")
    json.dump(detection_log, f, indent=2)
    f.write(";")
out.release()
cv2.destroyAllWindows()



0: 384x640 2 cars, 6 traffic lights, 138.1ms
Speed: 7.3ms preprocess, 138.1ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 6 traffic lights, 150.9ms
Speed: 9.5ms preprocess, 150.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 6 traffic lights, 108.6ms
Speed: 4.3ms preprocess, 108.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 6 traffic lights, 120.4ms
Speed: 5.5ms preprocess, 120.4ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 6 traffic lights, 113.5ms
Speed: 3.3ms preprocess, 113.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 6 traffic lights, 123.9ms
Speed: 4.9ms preprocess, 123.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 6 traffic lights, 105.4ms
Speed: 3.3ms preprocess, 105.4ms inference, 1.1ms postprocess per image at shape (1, 3, 38

In [5]:
def ccw(A, B, C):
    # Kiểm tra ba điểm có theo chiều ngược kim đồng hồ không
    return (C[1]-A[1]) * (B[0]-A[0]) > (B[1]-A[1]) * (C[0]-A[0])

def lines_intersect(A, B, C, D):
    # Kiểm tra đoạn AB có giao với đoạn CD không
    return ccw(A, C, D) != ccw(B, C, D) and ccw(A, B, C) != ccw(A, B, D)

def bbox_intersects_line(bbox, line):
    x1, y1, x2, y2 = bbox
    corners = [
        (x1, y1), (x2, y1),  # Top edge
        (x2, y2), (x1, y2)   # Bottom edge
    ]
    edges = [
        (corners[0], corners[1]),
        (corners[1], corners[2]),
        (corners[2], corners[3]),
        (corners[3], corners[0])
    ]
    for edge in edges:
        if lines_intersect(edge[0], edge[1], line[0], line[1]):
            return True
    return False



In [6]:
print("✅ Đoạn thẳng đã chọn:", line)

✅ Đoạn thẳng đã chọn: ((1176, 763), (691, 758))


In [7]:
import json

# Đọc và xử lý file detection_log.js
with open("detection_log.js", "r") as f:
    js_content = f.read()

# Xoá phần định nghĩa biến JS và dấu chấm phẩy cuối
js_content = js_content.replace("const detectionData = ", "").rstrip(";")

# Chuyển thành đối tượng Python
detection_data = json.loads(js_content)

# Danh sách lưu frame vi phạm
violating_frames = []

# Kiểm tra từng frame
for item in detection_data:
    if item.get("traffic_light", {}).get("color", [None, ""])[1] == "red":
        for vehicle in item.get("vehicles", []):
            bbox = (vehicle["x1"], vehicle["y1"], vehicle["x2"], vehicle["y2"])
            if bbox_intersects_line(bbox, line):
                violating_frames.append(item["frame"])
                break  # Một xe vi phạm là đủ ghi nhận frame đó

# In kết quả
print("Các frame vi phạm:", violating_frames)
       



Các frame vi phạm: [247, 248, 249, 250, 251, 353, 354, 366, 374, 379, 382, 387, 388, 390, 391, 393, 394]


In [8]:
import cv2

# Đầu vào
input_path = "end.mp4"
output_path = "real.mp4"

# Mở video
cap = cv2.VideoCapture(input_path)

# Lấy thông số video gốc
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Thiết lập ghi video đầu ra
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_index = 0
frame_list_set = set(violating_frames)  # Tăng tốc tìm kiếm

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_index in frame_list_set:
        out.write(frame)

    frame_index += 1

cap.release()
out.release()

print(f"✅ Đã xuất xong video gồm {len(violating_frames)} frame tại: {output_path}")


✅ Đã xuất xong video gồm 17 frame tại: real.mp4
